In [1]:
#Data Wrangling on Real Estate Market

In [1]:
import pandas as pd
import seaborn as sns


In [2]:
df = pd.read_csv("C:\Datasetsml\Bengaluru_House_Data.csv")
df.columns = df.columns.str.replace(" ", "_").str.replace("[^A-Za-z0-9_]", "")
df.head()


,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


In [3]:
# Load the 'tips' dataset from seaborn to simulate merging
tips = sns.load_dataset("tips")
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [4]:
# Add a sample 'location' column in the tips dataset for demonstration purposes
tips['location'] = tips['day']

In [5]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size,location
0,16.99,1.01,Female,No,Sun,Dinner,2,Sun
1,10.34,1.66,Male,No,Sun,Dinner,3,Sun
2,21.01,3.50,Male,No,Sun,Dinner,3,Sun
3,23.68,3.31,Male,No,Sun,Dinner,2,Sun
4,24.59,3.61,Female,No,Sun,Dinner,4,Sun


In [6]:
# Perform a left merge on the 'location' column
merged_data = pd.merge(df, tips[['total_bill', 'tip', 'location']], on="location", how="left")

In [7]:
# Display merged data
print(merged_data.head())

              area_type   availability                  location       size  \
0  Super built-up  Area         19-Dec  Electronic City Phase II      2 BHK   
1            Plot  Area  Ready To Move          Chikka Tirupathi  4 Bedroom   
2        Built-up  Area  Ready To Move               Uttarahalli      3 BHK   
3  Super built-up  Area  Ready To Move        Lingadheeranahalli      3 BHK   
4  Super built-up  Area  Ready To Move                  Kothanur      2 BHK   

   society total_sqft  bath  balcony   price  total_bill  tip  
0  Coomee        1056   2.0      1.0   39.07         NaN  NaN  
1  Theanmp       2600   5.0      3.0  120.00         NaN  NaN  
2      NaN       1440   2.0      3.0   62.00         NaN  NaN  
3  Soiewre       1521   3.0      1.0   95.00         NaN  NaN  
4      NaN       1200   2.0      1.0   51.00         NaN  NaN  


In [8]:
# Checking missing values
print(df.isnull().sum())

area_type          0
availability       0
location           1
size              16
society         5502
total_sqft         0
bath              73
balcony          609
price              0
dtype: int64


In [9]:
# Example strategy: Fill missing values in 'bath' and 'balcony' with median
df['bath'].fillna(df['bath'].median(), inplace=True)
df['balcony'].fillna(df['balcony'].median(), inplace=True)

C:\Users\asus\AppData\Local\Temp\ipykernel_7428\1674595655.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['bath'].fillna(df['bath'].median(), inplace=True)
C:\Users\asus\AppData\Local\Temp\ipykernel_7428\1674595655.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exa

In [10]:
# Drop rows with remaining missing values
df.dropna(inplace=True)

In [11]:
# Example: Filter for properties available in "Whitefield"
subset_data = df[df['location'] == 'Whitefield']
subset_data.head()


,area_type,availability,location,size,society,total_sqft,bath,balcony,price
5,Super built-up Area,Ready To Move,Whitefield,2 BHK,DuenaTa,1170,2.0,1.0,38.00
11,Plot Area,Ready To Move,Whitefield,4 Bedroom,Prrry M,2785,5.0,3.0,295.00
47,Super built-up Area,20-Sep,Whitefield,2 BHK,Goted U,1459,2.0,1.0,94.82
52,Built-up Area,Ready To Move,Whitefield,3 BHK,Suent V,2010,3.0,2.0,91.00
62,Plot Area,Ready To Move,Whitefield,4 Bedroom,Chranya,5700,5.0,3.0,650.00


In [12]:
# One-hot encoding 'area_type'
df = pd.get_dummies(df, columns=['area_type'], drop_first=True)


In [13]:
# Aggregating data by location
avg_price_location = df.groupby('location')['price'].mean().reset_index()


In [14]:
# Example using IQR for 'price' column
Q1 = df['price'].quantile(0.25)
Q3 = df['price'].quantile(0.75)
IQR = Q3 - Q1

# Filter out outliers
df = df[(df['price'] >= Q1 - 1.5 * IQR) & (df['price'] <= Q3 + 1.5 * IQR)]
